In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-01 15:11:31--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M  4.34MB/s    in 0.3s    

2023-10-01 15:11:32 (4.34 MB/s) - ‘data.csv’ saved [1475504/1475504]



In [35]:
df = pd.read_csv('data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [38]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.rename(columns = {'msrp':'price'}, inplace = True)
df = df.fillna(0).copy()

In [39]:
df.engine_fuel_type.value_counts()

engine_fuel_type
regular unleaded                                7172
premium unleaded (required)                     2009
premium unleaded (recommended)                  1523
flex-fuel (unleaded/E85)                         899
diesel                                           154
electric                                          66
flex-fuel (premium unleaded required/E85)         54
flex-fuel (premium unleaded recommended/E85)      26
flex-fuel (unleaded/natural gas)                   6
0                                                  3
natural gas                                        2
Name: count, dtype: int64

In [29]:
relevant_columns = [i.lower().replace(' ', '_') for i in 
        ["Make", "Model", "Year", "Engine HP", "Engine Cylinders", 
                    "Transmission Type", "Vehicle Style", "highway MPG", "city mpg"]]

In [12]:
df_relevant = df[relevant_columns].copy()

In [13]:
df_relevant

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18
...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16


In [14]:
df_relevant.transmission_type.value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

In [59]:
numerical_features = df_relevant.dtypes[(df_relevant.dtypes!='object')].index.to_list()
categorical_features = df_relevant.dtypes[(df_relevant.dtypes=='object')].index.to_list()
numerical_features, categorical_features

(['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg'],
 ['make', 'model', 'transmission_type', 'vehicle_style'])

In [45]:
for col in numerical_features:
    print(col)
    print(df_relevant[numerical_features].corrwith(df_relevant[col]).abs())
    print('----')

year
year                1.000000
engine_hp           0.351794
engine_cylinders    0.041479
highway_mpg         0.258240
city_mpg            0.198171
dtype: float64
----
engine_hp
year                0.351794
engine_hp           1.000000
engine_cylinders    0.779988
highway_mpg         0.406563
city_mpg            0.439371
dtype: float64
----
engine_cylinders
year                0.041479
engine_hp           0.779988
engine_cylinders    1.000000
highway_mpg         0.621606
city_mpg            0.600776
dtype: float64
----
highway_mpg
year                0.258240
engine_hp           0.406563
engine_cylinders    0.621606
highway_mpg         1.000000
city_mpg            0.886829
dtype: float64
----
city_mpg
year                0.198171
engine_hp           0.439371
engine_cylinders    0.600776
highway_mpg         0.886829
city_mpg            1.000000
dtype: float64
----


In [42]:
df_relevant[numerical_features].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.351794,-0.041479,0.258240,0.198171
engine_hp,0.351794,1.000000,0.779988,-0.406563,-0.439371
engine_cylinders,-0.041479,0.779988,1.000000,-0.621606,-0.600776
highway_mpg,0.258240,-0.406563,-0.621606,1.000000,0.886829
city_mpg,0.198171,-0.439371,-0.600776,0.886829,1.000000


## Make Price Binary

In [49]:
_mean = df.price.mean()
df['above_average'] = (df['price']>_mean).astype(int)

In [51]:
from sklearn.model_selection import train_test_split

In [92]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

y_full_train = df_full_train.above_average
y_train = df_train.above_average
y_test = df_test.above_average
y_val = df_val.above_average

In [60]:
df_full_train[categorical_features].nunique()

make                  48
model                902
transmission_type      5
vehicle_style         16
dtype: int64

In [93]:
from sklearn.metrics import mutual_info_score

for var in categorical_features:
    print('----')
    print(var)
    print(mutual_info_score(df_full_train.above_average, df_full_train[var]))

----
make
0.2387236479073192
----
model
0.46099440756035703
----
transmission_type
0.020883575914971135
----
vehicle_style
0.08339022741593435


Transmission type has the lowest mutual information score

In [94]:
df_full_train = df_full_train[relevant_columns].copy()
df_train = df_train[relevant_columns].copy()
df_test = df_test[relevant_columns].copy()
df_val = df_val[relevant_columns].copy()

In [95]:
from sklearn.feature_extraction import DictVectorizer

In [96]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [97]:
from sklearn.linear_model import LogisticRegression

In [98]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [99]:
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [100]:
dicts_test = df_test.to_dict(orient='records')

In [101]:
X_test = dv.transform(dicts_test)

In [102]:
y_pred = model.predict_proba(X_val)[:, 1]


In [103]:
above_average_price = (y_pred >= 0.5).astype(int)
above_average_price

array([0, 1, 0, ..., 0, 1, 1])

In [107]:
from sklearn.metrics import accuracy_score
original_accuracy = accuracy_score(y_val, above_average_price)
original_accuracy

0.9441879983214435

## Feature elemination

In [120]:
for feature in relevant_columns:
    feature_list = [i for i in relevant_columns if i!=feature]
    
    df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

    y_full_train = df_full_train.above_average
    y_train = df_train.above_average
    y_test = df_test.above_average
    y_val = df_val.above_average
    
    df_full_train = df_full_train[feature_list].copy()
    df_train = df_train[feature_list].copy()
    df_test = df_test[feature_list].copy()
    df_val = df_val[feature_list].copy()
    
    train_dict = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    
    above_average_price = (y_pred >= 0.5).astype(int)
    
    print(feature, accuracy_score(y_val, above_average_price), original_accuracy_score(y_val, above_average_price))

make 0.9429290809903483 0.0012589173310951773
model 0.9194292908099034 0.02475870751154008
year 0.9479647503147294 -0.003776751993285865
engine_hp 0.9227864036928242 0.021401594628619347
engine_cylinders 0.9454469156525388 -0.0012589173310952884
transmission_type 0.9458665547629039 -0.0016785564414604215
vehicle_style 0.9320184641208561 0.012169534200587417
highway_mpg 0.9475451112043642 -0.003357112882920732
city_mpg 0.9324381032312211 0.011749895090222395


## Linear Regression

In [130]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [132]:
alpha_values = [0, 0.01, 0.1, 1, 10]

for alpha in alpha_values:
    model = Ridge(solver='sag', alpha=alpha, random_state=42)
    
    df['price_log1p'] = df['price'].apply(lambda x: np.log1p(x))
    
    df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

    y_full_train = df_full_train.price_log1p
    y_train = df_train.price_log1p.values
    y_test = df_test.price_log1p.values
    y_val = df_val.price_log1p.values
    
    train_dict = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    print(alpha, mean_squared_error(y_val, y_pred))

0 0.9075003859646015
0.01 0.9075003859646015
0.1 0.9075003859646015
1 0.9075003859646015
10 0.9075003859685686


In [133]:
y_pred

array([ 9.97377621, 10.39145448,  9.85661718, ...,  9.88977766,
       11.00750128, 10.26751865])